----
#import

In [1]:
import math
import numpy as np
import itertools
import os
from tqdm import tqdm
from multiprocessing import Pool

----
#クラスの定義

In [2]:
class Field:
    
    '''
    :シングルトンクラス
    '''
    __instance = None

    @staticmethod 
    def getInstance():
        if Field.__instance == None:
            Field()
        return Field.__instance

    def __init__(self):
        if Field.__instance != None:
            raise Exception("Singletonクラス")
        else:
            Field.__instance = self

    def search(self,x,field):
        return field[x]

    def toTicTacToe(self,field):
        tictactoe = np.zeros(9,dtype=int)
        for i in range(9):
            if field[i] != 0:
                tictactoe[i] = field[i]
            elif field[i + 9] != 0:
                tictactoe[i] = field[i + 9]
            elif field[i + 18] != 0:
                tictactoe[i] = field[i + 18]
        return tictactoe

    def isWhiteWon(self,field):
        tictactoe = self.toTicTacToe(field)
        for i in range(3):
            if self.search(3*i,tictactoe) == self.search(3*i + 1,tictactoe) and self.search(3*i + 1,tictactoe) == self.search(3*i + 2,tictactoe) and self.search(3*i + 2,tictactoe) == 1:
                return True
            elif self.search(i,tictactoe) == self.search(i + 3,tictactoe) and self.search(i + 3,tictactoe) == self.search(i + 6,tictactoe) and self.search(i + 6,tictactoe) == 1:
                return True
        if self.search(0,tictactoe) == self.search(4,tictactoe) and self.search(4,tictactoe) == self.search(8,tictactoe) and self.search(8,tictactoe) == 1:
            return True
        if self.search(2,tictactoe) == self.search(4,tictactoe) and self.search(4,tictactoe) == self.search(6,tictactoe) and self.search(6,tictactoe) == 1:
            return True
        return False

    def isBlackWon(self,field):
        tictactoe = self.toTicTacToe(field)
        for i in range(3):
            if self.search(3*i,tictactoe) == self.search(3*i + 1,tictactoe) and self.search(3*i + 1,tictactoe) == self.search(3*i + 2,tictactoe) and self.search(3*i + 2,tictactoe) == 2:
                return True
            elif self.search(i,tictactoe) == self.search(i + 3,tictactoe) and self.search(i + 3,tictactoe) == self.search(i + 6,tictactoe) and self.search(i + 6,tictactoe) == 2:
                return True
        if self.search(0,tictactoe) == self.search(4,tictactoe) and self.search(4,tictactoe) == self.search(8,tictactoe) and self.search(8,tictactoe) == 2:
            return True
        if self.search(2,tictactoe) == self.search(4,tictactoe) and self.search(4,tictactoe) == self.search(6,tictactoe) and self.search(6,tictactoe) == 2:
            return True
        return False

    def isAble(self,field):
        '''
          :param field: 局面
          :return: boolean
        '''
        countWhite = 0
        countBlack = 0

        for i in range(0,9):
            if (field[i] == 1):
                countWhite += 1
            elif (field[i] == 2):
                countBlack += 1
        if (countWhite>2 or countBlack>2):
            return False
        else:
            countWhite = 0;
            countBlack = 0;

        for i in range(9,18):
            if (field[i] == 1):
                countWhite += 1
            elif (field[i] == 2):
                countBlack += 1

        if (countWhite>2 or countBlack>2):
            return False
        else:
            countWhite = 0
            countBlack = 0

        for i in range(18,27):
            if (field[i] == 1):
                countWhite += 1
            elif (field[i] == 2):
                countBlack += 1
        if (countWhite>2 or countBlack>2):
            return False
        else:
            countWhite = 0
            countBlack = 0

        return True

    def Base_10_to_n(self,X,n):
        '''
          :param X: 10進数
          :param n: n進数
          :return: 10進数をn進数に変換したもの
        '''
        if (int(X/n)):
            return self.Base_10_to_n(int(X/n), n)+str(X%n)

        return str(X%n)

    def radixConversionFromTen(self,x):
        '''
          :param x: 10進数
          :return: 3進数、要素27の配列
        '''
        word = self.Base_10_to_n(x,3)
        word_list = list(word)
        while len(word_list) != 27:
          word_list.insert(0,"0")
        num_list = []
        for s in word_list:
          num_list.append(int(s))

        return num_list;

    def radixConversionToTen(self,field,x):
        code = 0
        for i in range(len(field)):
            code += field[i] * (x**i)
        return code

In [3]:
class AllPositions:

    def __init__(self):
        self.allpositions = []
        self.hashcode = []
        self.field = Field.getInstance()

    def listUpAllPositions(self):
        for count_white in range (3):
            for var_white in range (self.combinations_count(9,count_white)):
                layer = np.zeros(9,dtype=int)
                white_placed = self.placement(9,count_white,var_white,layer,1)

                for count_black in range(3):
                    for var_black in range (self.combinations_count(9-count_white,count_black)):
                        black_placed = self.placement(9,count_black,var_black,white_placed,2)
                        if len(black_placed) != 0:
                            self.allpositions.append(list(black_placed))
            

    def combinations_count(self,n,r):
        return math.factorial(n) // (math.factorial(n - r) * math.factorial(r))

    def placement(self,squares,count,var,prelayer,white_or_black):
        mylist = []
        for i in range(len(prelayer)):
            if prelayer[i] == 0:
                mylist.append(int(i))
        balls = itertools.combinations(mylist,count)
        placements = list(balls)[var]
        layer = list(prelayer)
        for num in placements:
            layer[num] = white_or_black
        return layer

    def getPositionsFromId(self,id):
        if type(id) is not str:
            id = str(id)
        if ((len(self.getAllPositions()) != 0) and (len(id) == 12)):
            lst = list(id)
            a = lst[0]+ lst[1] + lst[2] + lst[3]
            b = lst[4]+ lst[5] + lst[6] + lst[7]
            c = lst[8]+ lst[9] + lst[10] + lst[11]
            field = []
            field += self.getAllPositions()[int(a)] + self.getAllPositions()[int(b)] + self.getAllPositions()[int(c)]
            return field
        return []

    def getLayerFromId(self,id):
        if type(id) is not str:
            id = str(id).zfill(4)
        if ((len(self.getAllPositions()) != 0) and (len(id) == 4)):
            lst = list(id)
            a = lst[0]+ lst[1] + lst[2] + lst[3]
            field = []
            field += self.getAllPositions()[int(a)]
            return field
        return [] 

    def createHashcode(self,field):
        hashlist = []
        for i in range(len(field)):
            code = 0
            for j in range(9):
                code += field[i][j] * (3**j)
            hashlist.append(code)
        self.hashcode = hashlist

    def getAllPositions(self):
        return self.allpositions

    def getHashcode(self):
        return self.hashcode

In [4]:
field = Field.getInstance()

allPositions = AllPositions()
allPositions.listUpAllPositions()
layer1 = allPositions.getAllPositions()
layer2 = list(layer1)
layer3 = list(layer1)
print(layer1)
print(len(layer1))
print(allPositions.getPositionsFromId(100909760653))

[[0, 0, 0, 0, 0, 0, 0, 0, 0], [2, 0, 0, 0, 0, 0, 0, 0, 0], [0, 2, 0, 0, 0, 0, 0, 0, 0], [0, 0, 2, 0, 0, 0, 0, 0, 0], [0, 0, 0, 2, 0, 0, 0, 0, 0], [0, 0, 0, 0, 2, 0, 0, 0, 0], [0, 0, 0, 0, 0, 2, 0, 0, 0], [0, 0, 0, 0, 0, 0, 2, 0, 0], [0, 0, 0, 0, 0, 0, 0, 2, 0], [0, 0, 0, 0, 0, 0, 0, 0, 2], [2, 2, 0, 0, 0, 0, 0, 0, 0], [2, 0, 2, 0, 0, 0, 0, 0, 0], [2, 0, 0, 2, 0, 0, 0, 0, 0], [2, 0, 0, 0, 2, 0, 0, 0, 0], [2, 0, 0, 0, 0, 2, 0, 0, 0], [2, 0, 0, 0, 0, 0, 2, 0, 0], [2, 0, 0, 0, 0, 0, 0, 2, 0], [2, 0, 0, 0, 0, 0, 0, 0, 2], [0, 2, 2, 0, 0, 0, 0, 0, 0], [0, 2, 0, 2, 0, 0, 0, 0, 0], [0, 2, 0, 0, 2, 0, 0, 0, 0], [0, 2, 0, 0, 0, 2, 0, 0, 0], [0, 2, 0, 0, 0, 0, 2, 0, 0], [0, 2, 0, 0, 0, 0, 0, 2, 0], [0, 2, 0, 0, 0, 0, 0, 0, 2], [0, 0, 2, 2, 0, 0, 0, 0, 0], [0, 0, 2, 0, 2, 0, 0, 0, 0], [0, 0, 2, 0, 0, 2, 0, 0, 0], [0, 0, 2, 0, 0, 0, 2, 0, 0], [0, 0, 2, 0, 0, 0, 0, 2, 0], [0, 0, 2, 0, 0, 0, 0, 0, 2], [0, 0, 0, 2, 2, 0, 0, 0, 0], [0, 0, 0, 2, 0, 2, 0, 0, 0], [0, 0, 0, 2, 0, 0, 2, 0, 0], [0, 0, 0, 2, 

In [5]:
layer1

[[0, 0, 0, 0, 0, 0, 0, 0, 0],
 [2, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 2, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 2, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 2, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 2, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 2, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 2, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 2, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 2],
 [2, 2, 0, 0, 0, 0, 0, 0, 0],
 [2, 0, 2, 0, 0, 0, 0, 0, 0],
 [2, 0, 0, 2, 0, 0, 0, 0, 0],
 [2, 0, 0, 0, 2, 0, 0, 0, 0],
 [2, 0, 0, 0, 0, 2, 0, 0, 0],
 [2, 0, 0, 0, 0, 0, 2, 0, 0],
 [2, 0, 0, 0, 0, 0, 0, 2, 0],
 [2, 0, 0, 0, 0, 0, 0, 0, 2],
 [0, 2, 2, 0, 0, 0, 0, 0, 0],
 [0, 2, 0, 2, 0, 0, 0, 0, 0],
 [0, 2, 0, 0, 2, 0, 0, 0, 0],
 [0, 2, 0, 0, 0, 2, 0, 0, 0],
 [0, 2, 0, 0, 0, 0, 2, 0, 0],
 [0, 2, 0, 0, 0, 0, 0, 2, 0],
 [0, 2, 0, 0, 0, 0, 0, 0, 2],
 [0, 0, 2, 2, 0, 0, 0, 0, 0],
 [0, 0, 2, 0, 2, 0, 0, 0, 0],
 [0, 0, 2, 0, 0, 2, 0, 0, 0],
 [0, 0, 2, 0, 0, 0, 2, 0, 0],
 [0, 0, 2, 0, 0, 0, 0, 2, 0],
 [0, 0, 2, 0, 0, 0, 0, 0, 2],
 [0, 0, 0, 2, 2, 0, 0, 0, 0],
 [0, 0, 0, 2, 0, 2, 0, 0, 0],
 [0, 0, 0,

----
#白の勝利局面の算出

In [ ]:
os.chdir("/content/drive/MyDrive/RetrogradeAnalysis/BlackLosed")

In [ ]:
for i in tqdm(range(len(layer1)),total=len(layer1)):
    blackLosedPosition = []
    for j in range(len(layer2)):
        for k in range(len(layer3)):
            id = str(i).zfill(4) + str(j).zfill(4) + str(k).zfill(4)
            position = allPositions.getPositionsFromId(id)
            if field.isWhiteWon(position):
                blackLosedPosition.append(id)
    blackLosedPosition = np.array(blackLosedPosition)
    np.savetxt(f'BlackLosed_layer1={str(i).zfill(4)}.csv',blackLosedPosition,delimiter=",",fmt="%s")

 44%|████▎     | 622/1423 [6:28:36<8:06:58, 36.48s/it]

----
#白の勝利局面の１手前の局面の算出


In [6]:
#for i in tqdm(range(len(layer1)),total=len(layer1)):
def whitePreviousPosition_1(i):
    whitePreviousPosition = []
    os.chdir("/content/drive/MyDrive/RetrogradeAnalysis/BlackLosed")
    position = np.loadtxt(f'BlackLosed_layer1={str(i).zfill(4)}.csv',delimiter=",").astype(int).astype(str).tolist()
    os.chdir("/content/drive/MyDrive/RetrogradeAnalysis/WhitePrevious-1")
    if not(os.path.isfile(f'WhitePrevious_layer1={str(i).zfill(4)}.csv')):
        for j in range(len(position)):
            id = position[j].zfill(12)
            target = allPositions.getPositionsFromId(id)
            if 2 in target:
                for k in range(len(target)):
                    delete_from_target = list(target)
                    move_from_target = list(target)
                    if target[k] == 1:
                        delete_from_target[k] = 0
                        move_from_target[k] = 0
                        if not(field.isWhiteWon(delete_from_target)) and (field.isAble(delete_from_target)):
                            del_target = field.radixConversionToTen(delete_from_target,3)
                            whitePreviousPosition.append(del_target)
                        for l in range(len(target)):
                            if k != l:
                                move_from_target_cloned = list(move_from_target)
                                move_from_target_cloned[l] = 1
                                if not(field.isWhiteWon(move_from_target_cloned)) and (field.isAble(move_from_target_cloned)):
                                    mov_target = field.radixConversionToTen(move_from_target_cloned,3)
                                    whitePreviousPosition.append(mov_target)
        whitePreviousPosition = list(set(whitePreviousPosition))
        print(len(whitePreviousPosition))
        np.savetxt(f'WhitePrevious_layer1={str(i).zfill(4)}.csv',whitePreviousPosition,delimiter=",",fmt="%s")
        return len(whitePreviousPosition)

In [ ]:
Pool(2).map(whitePreviousPosition_1,range(len(layer1)))

5874512
3161744
7714455
6111729
7091464
3781516
6157095
7475558
6359521
4331480
7922627
6184375
3659819
3929250
3833612
7399214
5974065


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
